In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import keras_tuner as kt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras. preprocessing.sequence import pad_sequences

In [2]:
train = pd.read_csv('C:/Users/mandy/OneDrive/2022 IRS/ML/Datasets/Chinese Word Segmentation/4. Chinese Word Segmentation/as_training.utf8', names=['data'])
print(train.head())

                    data
0                   時間　：
1  三月　十日　（　星期四　）　上午　十時　。
2                   地點　：
3      學術　活動　中心　一樓　簡報室　。
4                   主講　：


In [3]:
y = train['data'].copy()
X = train['data'].str.replace('\u3000', '')
print(X.head())
print(type(X))
print(y.head())

0               時間：
1    三月十日（星期四）上午十時。
2               地點：
3      學術活動中心一樓簡報室。
4               主講：
Name: data, dtype: object
<class 'pandas.core.series.Series'>
0                     時間　：
1    三月　十日　（　星期四　）　上午　十時　。
2                     地點　：
3        學術　活動　中心　一樓　簡報室　。
4                     主講　：
Name: data, dtype: object


In [ ]:
# labels = [[] * len(y)]
# for index, line in enumerate(y.data):
#     for char_index in range(len(line) - 1):
#         if char_index == len(line) - 1:
#             labels[index].append(1)
#         elif line[char_index + 1] == '\u3000':
#             labels[index].append(1)
#         else:
#             labels[index].append(0)

# print(labels)

In [4]:
def create_labels(data):
    label = []
    index = 0

    while index < len(data) - 1:
        if data[index + 1] == '\u3000':
            label.append(1)
            index += 2
        else:
            label.append(0)
            index += 1
            
    if index == len(data) - 1:
        label.append(1)
    return label

y.apply(create_labels)

0                                          [0, 1, 1]
1         [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1]
2                                          [0, 1, 1]
3               [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]
4                                          [0, 1, 1]
                             ...                    
708948             [0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1]
708949                      [0, 1, 1, 1, 0, 0, 1, 1]
708950                                     [0, 1, 1]
708951    [0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1]
708952          [0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1]
Name: data, Length: 708953, dtype: object